<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Create-reports-for-each-inspection-batch" data-toc-modified-id="Create-reports-for-each-inspection-batch-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create reports for each inspection batch</a></span></li><li><span><a href="#Analysis-of-inspection-batches" data-toc-modified-id="Analysis-of-inspection-batches-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Analysis of inspection batches</a></span></li><li><span><a href="#Error-analysis" data-toc-modified-id="Error-analysis-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Error analysis</a></span><ul class="toc-item"><li><span><a href="#Histogram-of-the-job_duration-batches" data-toc-modified-id="Histogram-of-the-job_duration-batches-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Histogram of the <code>job_duration</code> batches</a></span></li><li><span><a href="#Boxplot-of-the-job_duration-batches" data-toc-modified-id="Boxplot-of-the-job_duration-batches-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Boxplot of the <code>job_duration</code> batches</a></span></li><li><span><a href="#Boxplot-of-the-elapsed-time-batches" data-toc-modified-id="Boxplot-of-the-elapsed-time-batches-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Boxplot of the <code>elapsed time</code> batches</a></span></li><li><span><a href="#Boxplot-of-the-rate-batches" data-toc-modified-id="Boxplot-of-the-rate-batches-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Boxplot of the <code>rate</code> batches</a></span></li></ul></li><li><span><a href="#Interpolations" data-toc-modified-id="Interpolations-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Interpolations</a></span><ul class="toc-item"><li><span><a href="#Box-Plots-interpolated-for-job_duration" data-toc-modified-id="Box-Plots-interpolated-for-job_duration-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Box Plots interpolated for <code>job_duration</code></a></span></li><li><span><a href="#Box-Plots-interpolated-for-elapsed_time" data-toc-modified-id="Box-Plots-interpolated-for-elapsed_time-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Box Plots interpolated for <code>elapsed_time</code></a></span></li><li><span><a href="#Box-Plots-interpolated-for-rate" data-toc-modified-id="Box-Plots-interpolated-for-rate-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Box Plots interpolated for <code>rate</code></a></span></li></ul></li><li><span><a href="#Inspection-Time-analysis" data-toc-modified-id="Inspection-Time-analysis-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Inspection Time analysis</a></span><ul class="toc-item"><li><span><a href="#Inspection-Builds-Time" data-toc-modified-id="Inspection-Builds-Time-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Inspection Builds Time</a></span></li><li><span><a href="#Inspection-Jobs-Time" data-toc-modified-id="Inspection-Jobs-Time-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Inspection Jobs Time</a></span></li><li><span><a href="#Inspection-Builds-+-Jobs-Time" data-toc-modified-id="Inspection-Builds-+-Jobs-Time-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Inspection Builds + Jobs Time</a></span></li></ul></li></ul></div>

# Introduction

In [ ]:
from thoth.lab import inspection, inspection_report, underscore

import logging
logger = logging.getLogger()
logger.handlers = []

logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)

In [ ]:
%env THOTH_DEPLOYMENT_NAME     thoth-core-psi-stage
%env THOTH_CEPH_BUCKET         thoth
%env THOTH_CEPH_BUCKET_PREFIX  data/thoth
%env THOTH_S3_ENDPOINT_URL     https://s3.upshift.redhat.com/

In [ ]:
identifier_inspection = [
    "64-matrix-size",
    "128-matrix-size",
    "256-matrix-size",
    "test-ms", # 512-matrix-size
    "1024-matrix-size",
    "2048-matrix-size",
    "4096-matrix-size"
]

static_figure=True
limit_results=False

In [ ]:
parameters_analysis = [
    "job_duration",
    "build_duration",
    "elapsed",
    "rate"
]

In [ ]:
filtered_inspection_ids = inspection.filter_inspection_ids(identifier_inspection)

In [ ]:
inspection_results_dict = inspection.aggregate_inspection_results_per_identifier(filtered_inspection_ids,
                                                                                identifier_inspection,
                                                                                  limit_results=limit_results)

In [ ]:
inspection_df_dict = inspection.create_inspection_dataframes(inspection_results_dict)

# Create reports for each inspection batch

In [ ]:
# Collect all reports and provide results
inspection_report_dict = inspection_report.create_inspection_reports(inspection_df_dict)

# Analysis of inspection batches

In [ ]:
inspection.create_inspection_analysis_plots(inspection_df_dict["64-matrix-size"])

In [ ]:
inspection_report.create_feature_analysis_summary(inspection_report_dict)

In [ ]:
inspection_report.create_feature_analysis_summary(inspection_report_dict, explanation=True)

# Error analysis

In [ ]:
inspection_parameters_df_dict = inspection.create_inspection_parameters_dataframes(
    parameters_analysis,
    inspection_df_dict)

## Histogram of the `job_duration` batches

In [ ]:
inspection.create_plot_multiple_batches(inspection_parameters_df_dict["job_duration"],
                plot_type="hist",
                x_label="Inspection Time [s]",
                y_label="Count",
                plot_title="Job duration distribution per batch",
                static=static_figure)

## Boxplot of the `job_duration` batches

In [ ]:
inspection.create_box_plot_multiple_columns(inspection_parameters_df_dict["job_duration"],
                plot_type="box",
                x_label="Batch identifier",
                y_label="Job Duration Time [s]",
                plot_title="Box plots job duration per batch",
                static=static_figure)

## Boxplot of the `elapsed time` batches

In [ ]:
inspection.create_box_plot_multiple_columns(inspection_parameters_df_dict["elapsed"],
                plot_type="box",
                x_label="Batch identifier",
                y_label="Elapsed Time [ms]",
                plot_title="Box plots elapsed time per batch",
                static=static_figure)

## Boxplot of the `rate` batches

In [ ]:
inspection.create_box_plot_multiple_columns(inspection_parameters_df_dict["rate"],
                plot_type="box",
                x_label="Batch identifier",
                y_label="Rate [GFLOPS]",
                plot_title="Box plots rate per batch",
                static=static_figure)

# Interpolations

In [ ]:
inspection_statistics_dict = inspection.evaluate_inspection_statistics(
    parameters_analysis,
    inspection_df_dict)

## Box Plots interpolated for `job_duration`

In [ ]:
parameter_selected = ["job_duration"]
colour = ["g", "b", "b", "r", "y", "k"]
quantities = ["median", "q1", "q3", "max", "min", "iqr"]

inspection.plot_interpolated_statistics_of_inspection_parameters(statistical_results_dict=inspection_statistics_dict,
                                                      identifier_inspection_list=identifier_inspection,
                                                      inspection_parameters=parameter_selected,
                                                      colour_list=colour,
                                                      statistical_quantities=quantities,
                                                      title_ylabel= "Job duration [s]")

## Box Plots interpolated for `elapsed_time`

In [ ]:
parameter_selected = ["elapsed"]
colour = ["g", "b", "b", "r", "y", "k"]
quantities = ["median", "q1", "q3", "max", "min", "iqr"]
inspection.plot_interpolated_statistics_of_inspection_parameters(statistical_results_dict=inspection_statistics_dict,
                                                      identifier_inspection_list=identifier_inspection,
                                                      inspection_parameters=parameter_selected,
                                                      colour_list=colour,
                                                      statistical_quantities=quantities,
                                                      title_ylabel= "Elapsed Time [ms]")

## Box Plots interpolated for `rate`

In [ ]:
parameter_selected = ["rate"]
colour = ["g", "b", "b", "r", "y", "k"]
quantities = ["median", "q1", "q3", "max", "min", "iqr"]
inspection.plot_interpolated_statistics_of_inspection_parameters(statistical_results_dict=inspection_statistics_dict,
                                                      identifier_inspection_list=identifier_inspection,
                                                      inspection_parameters=parameter_selected,
                                                      colour_list=colour,
                                                      statistical_quantities=quantities,
                                                      title_ylabel= "Rate [GFLOPS]")

# Inspection Time analysis

In [40]:
inspection_time_df = inspection.create_inspection_time_dataframe(inspection_df_dict)

## Inspection Builds Time

In [ ]:
inspection.create_plot_from_df(inspection_time_df, 
            columns= ["batches","builds_time"],
            title_plot="Time spent to evaluate all inspections builds in hours (ASSUMPTIONS ~6 in parallel)",
            x_label="Batch Identifier", 
            y_label="Tot Time for builds [h]",
            static=static_figure)

## Inspection Jobs Time

In [ ]:
inspection.create_plot_from_df(inspection_time_df, 
            columns= ["batches","jobs_time"],
            title_plot="Time spent to evaluate all inspections jobs in hours (ASSUMPTIONS ~6 in parallel)",
            x_label="Batch Identifier", 
            y_label="Tot Time for jobs [h]",
            static=static_figure)

## Inspection Builds + Jobs Time

In [ ]:
inspection.create_plot_from_df(inspection_time_df, 
            columns= ["batches","tot_time"],
            title_plot="Time spent to evaluate all inspections builds + jobs in hours (ASSUMPTIONS ~6 in parallel)",
            x_label="Batch Identifier", 
            y_label="Tot Time for builds + jobs [h]",
            static=static_figure)